## monitors

In [5]:
%matplotlib inline
import pandas as pd
import urllib
import urllib2
import time
import sys
import urllib
import urllib2
import re
import time
import platform
try:
    from colorama import init,Fore
except Exception, e:
    print '%s\n%s'%("I want module colorama.","Please try pip install colorama.")
    exit(0)
    
reload(sys)
sys.setdefaultencoding("utf-8")
os_type = str(platform.system())
init(autoreset=True)
colorlist = [Fore.RESET,Fore.LIGHTBLACK_EX,Fore.WHITE] 

# 用非 贪婪模式 匹配 \t 或者 \n 或者 空格 或者 超链接 或者 图片
BgnCharToNoneRex = re.compile("(\t|\n| |<a.*?>|<img.*?>)")

# 用非 贪婪模式 匹配 任意<>标签
EndCharToNoneRex = re.compile("<.*?>")

# 用非 贪婪模式 匹配 任意<p>标签
BgnPartRex = re.compile("<p.*?>")
CharToNewLineRex = re.compile("(<br/>|</p>|<tr>|<div>|</div>)")
CharToNextTabRex = re.compile("<td>")

# 将一些html的符号实体转变为原始符号
replaceTab = [("&lt;","<"),("&gt;",">"),("&amp;","&"),("&nbsp;"," ")]
resultplus = []
resultminer = []
def Replace_Char(x):
    x = BgnCharToNoneRex.sub("",x)
    x = BgnPartRex.sub("\n    ",x)
    x = CharToNewLineRex.sub("\n",x)
    x = CharToNextTabRex.sub("\t",x)
    x = EndCharToNoneRex.sub("",x)

    for t in replaceTab:  
        x = x.replace(t[0],t[1])  
    return x  

def GetPage(shortcode):
    user_agent = 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT)'
    headers = {'User-Agent' : user_agent}
    # Tencet API
    myUrl = 'http://qt.gtimg.cn/q=' + shortcode
    req = urllib2.Request(myUrl,headers = headers)
    try:
        myResponse = urllib2.urlopen(req)
        if os_type == 'Windows':
            UnicodePage = myResponse.read()
        else:
            UnicodePage = myResponse.read().decode('gbk').encode('utf-8')
    except:
        pass
    # 匹配股票数据
    myItems = re.findall('v_(.*?)=".*?~(.*?)~\d+~(.*?)~(.*?)~.*?;',UnicodePage,re.S)
    #print "          -----------%s-----------"%(time.asctime(time.localtime()))
    #print "        %-8s %-8s %-10s %-+6s %-s\n"%('Symbol','Name','Last Trade','Change','Chg')
    for result in myItems:
        # print result
        # print "###%s###\nname    %s\ncurr    %s\nperc    %.2f\n"%(result[0],result[1],result[2],float(result[2])-float(result[3]))
        delta = float(result[2])-float(result[3])
        perc = format(delta / float(result[3]),'.2%')
        if perc[0] != '-':
            perc = '+'+perc
        if delta > 0:
            flag = 0
        elif delta < 0 :
            flag = 1
        else :
            flag = 0
            
        result = [result[0],colorlist[flag]+result[1],perc]
        
        if perc[0] == '+':
            
            resultplus.append(result)
        else:
            resultminer.append(result)

while True:
    lines = open('list.txt','r').readlines()
    stocklist = []
    for line in lines:
        if line.startswith('6'):
            line = 'sh' + line
        else:
            line = 'sz' + line
        stocklist.append(line.replace('\n',''))
        GetPage(line.replace('\n',''))
    #print "          -----------%s-----------"%(time.asctime(time.localtime()))
    frameresultplus = pd.DataFrame(resultplus)
    frameresultplus.to_csv('resultplus.txt', index=False)
    frameresultminer = pd.DataFrame(resultminer)
    frameresultminer.to_csv('resultminer.txt', index=False)
    resultplus = []
    resultminer = []

    time.sleep(2)


IndentationError: expected an indented block (<ipython-input-5-b4c69f91d49a>, line 63)